# Table of Contents
<div class="toc" style="margin-top: 1em;">
   <ul class="toc-item" id="toc-level0">
<!--       <li><span><a href="#Overview" data-toc-modified-id="Overview"><span class="toc-item-num">I.&nbsp;&nbsp;</span>Overview of Tutorial</a></span></li>
      <li><span><a href="#Part-1" data-toc-modified-id="Part-1"><span class="toc-item-num">II.&nbsp;&nbsp;</span>Part 1</a></span></li>
      <ul class="toc-item" id="toc-level1">
         <li><span><a href="#Jupyter-Notebooks" data-toc-modified-id="Jupyter-Notebooks"><span class="toc-item-num">1.&nbsp;&nbsp;</span>Jupyter Notebooks</a></span></li>
         <li>
            <span><a href="#API-Overview" data-toc-modified-id="API-Overview-1"><span class="toc-item-num">2.&nbsp;&nbsp;</span>API Overview</a></span>
            <ul class="toc-item">
               <li><span><a href="#Context" data-toc-modified-id="Context-1.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Context</a></span></li>
               <li><span><a href="#Creating-a-ClipperConnection" data-toc-modified-id="Create-a-ClipperConnection-1.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Creating a ClipperConnection</a></span></li>
               <li><span><a href="#Starting-Clipper" data-toc-modified-id="Starting-Clipper-1.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Starting Clipper</a></span></li>
               <li>
                  <span><a href="#Deploying-a-model" data-toc-modified-id="Deploying-a-model-1.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Deploying a model</a></span>
                  <ul class="toc-item">
                     <li>
                        <span><a href="#Creating-a-model" data-toc-modified-id="Create-the-model-1.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Creating a model</a></span>
                     </li>
                     <li><span><a href="#Deploying-to-Clipper" data-toc-modified-id="Deploying-to-Clipper-1.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Deploying to Clipper</a></span></li>
                     <li><span><a href="#A-Note-About-Types-[Optional]" data-toc-modified-id="A-Note-About-Types-[Optional]-1.4.3"><span class="toc-item-num">2.4.3&nbsp;&nbsp;</span>A Note About Types [Optional]</a></span></li>
                  </ul>
               </li>
               <li><span><a href="#Registering-an-Application" data-toc-modified-id="Registering-an-Application-1.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Registering an Application</a></span></li>
               <li><span><a href="#Inspecting-Clipper" data-toc-modified-id="Inspecting-Clipper-1.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Inspecting Clipper</a></span></li>
               <li><span><a href="#Updating-the-Model" data-toc-modified-id="Updating-the-Model-1.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Updating the Model</a></span></li>
               <li><span><a href="#Adding-Model-Replicas" data-toc-modified-id="Adding-Model-Replicas-1.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Adding Model Replicas</a></span></li>
            </ul>
         </li>
      </ul>
   </ul> -->
      <ul class="toc-item" id="toc-level00">
          <li><span><a href="#Part-2" data-toc-modified-id="Part 2"><span class="toc-item-num">III.&nbsp;&nbsp;</span>Part 2</a></span></li>
         <ul class="toc-item" id="toc-level11">
            <li>
               <span><a href="#Example-Application---Image-Classification" data-toc-modified-id="Example-Application---Image-Classification"><span class="toc-item-num">3.&nbsp;&nbsp;</span>Example Application - Image Classification</a></span>
            <li><span><a href="#Example-Application---Custom-Docker-Containers" data-toc-modified-id="Example-Application---Custom-Docker-Containers"><span class="toc-item-num">4.&nbsp;&nbsp;</span>Example Application - Custom Docker Containers</a></span></li>
            <li><span><a href="#Restarting-Clipper" data-toc-modified-id="Restarting-Clipper"><span class="toc-item-num">5.&nbsp;&nbsp;</span>Restarting Clipper</a></span></li>
         </ul>
      </ul>
</div>

## Part 2

The following cell contains some initialization code. If you ran through [Part 1](1-Clipper-API.ipynb), then you can skip it.

In [ ]:
from clipper_admin import ClipperConnection, DockerContainerManager
import requests, json
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()
clipper_addr = clipper_conn.get_query_addr()

### Example Application - Image Classification

In the second part of this exercise, you will deploy a pre-trained [SqueezeNet](https://arxiv.org/abs/1602.07360) model that you will download from the PyTorch model zoo to classify images.

You will create an application that labels images from the ImageNet dataset.

Some sample images have already been downloaded for you.

#### Creating an application

For this tutorial, create an application named "squeezenet-classifier". Note that Clipper allows you to create the application before deploying any models.

In [ ]:
app_name = "squeezenet-classsifier"
default_output = "default"

clipper_conn.register_application(
    name=app_name,
    input_type="bytes",
    default_output=default_output,
    slo_micros=10000000)

When you list the applications registered with Clipper, you should see the newly registered "squeezenet-classifier" application show up.

In [ ]:
clipper_conn.get_all_apps()

#### Start serving

As soon as you register the application, the REST endpoint is live, even though no models have been linked yet. However, in this exercsie you will wait until you've deployed and linked a model before querying the application.

#### Import the pretrained model from the PyTorch model zoo

We will be using a pretrained SqueezeNet model from PyTorch. Download the SqueezeNet model from the PyTorch model zoo and import it into your notebook.

<details>
<summary>Details about the model</summary>
<p>Several common machine learning libraries, including PyTorch and TensorFlow, maintain a "model zoo" of pre-trained models that can be downloaded and used for inference without the need to train them. Most of the models in these model zoos are state-of-the-art deep-learning models which require an enormous amount of compute power (often several thousand GPU-hours) and expertise to train well. PyTorch provides a simple API to download and import models available in their computer vision model zoo.</p></details>

In [ ]:
from torchvision import models, transforms
model = models.squeezenet1_1(pretrained=True)

#### Deploying the PyTorch Model

Unlike the Scikit-Learn model in [Section 1](#API-Overview), PyTorch models cannot just be pickled and loaded. Instead, they must be saved using PyTorch's native serialization API. Because of this, you cannot use the generic Python model deployer to deploy the model to Clipper. Instead, you will use the Clipper PyTorch deployer to deploy it. The Docker container will load and reconstruct the model from the serialized model checkpoint when the container is started.

##### Preprocessing
Before making the predict function, you will download some labels for the dataset to test the accuracy of the pre-trained model, and specify some preprocessing logic to transform the images into the format the model expects. The code for this cell comes from the this [tutorial](http://blog.outcome.io/pytorch-quick-start-classifying-an-image/).

In [ ]:
# First we define the preproccessing on the images:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

# Then we download the labels:
labels = {int(key):value for (key, value)
          in requests.get('https://s3.amazonaws.com/outcome-blog/imagenet/labels.json').json().items()}

#### Define a predict function and add metrics
Now you can define a predict function to deploy. This predict function will wrap the model you downloaded from the model zoo as well as the preprocessing logic you just specified.

You will also use Clipper's metrics reporting functionality to export some custom metrics for this model. In particular, you will report the batch size for each batch sent to the model. You can view a simple metrics dashboard automatically exported by Prometheus at port 9090 (`http:://[IP_ADDRESS]:9090`). You will also get to see a detailed dashboard in Grafana.

<details>
<summary>More info on Metrics</summary>
<p>These metrics will be aggregated along with several system metrics and reported to [Prometheus](https://prometheus.io/), a time-series database commonly used for metrics reporting (for example, Kubernetes uses Prometheus for its own system metrics).</p>
<p>In practice, a monitoring and alerting system such as [Grafana](https://grafana.com/) is typically layered on top of Promtheus. You will work through an example of connecting Grafana to visualize Clipper's metrics later in the tutorial.</p></details>

In [ ]:
import clipper_admin.metrics as metrics

def predict_torch_model(model, imgs):
    import io
    import PIL.Image
    import torch
    import clipper_admin.metrics as metrics
    
    metrics.add_metric("batch_size", 'Gauge', 'Batch size passed to PyTorch predict function.')
    metrics.report_metric('batch_size', len(imgs))
    
    # We first prepare a batch from `imgs`
    img_tensors = []
    for img in imgs:
        img_tensor = preprocess(PIL.Image.open(io.BytesIO(img)))
        img_tensor.unsqueeze_(0)
        img_tensors.append(img_tensor)
    img_batch = torch.cat(img_tensors)
    
    # We perform a forward pass
    with torch.no_grad():
        model_output = model.eval()(img_batch)
        
    # Parse Result
    img_labels = [labels[out.data.numpy().argmax()] for out in model_output]
        
    return img_labels

> *Once again, Clipper must download this Docker image from the internet, so this may take a minute. Thanks for your patience.*

In [ ]:
from clipper_admin.deployers import pytorch as pytorch_deployer
pytorch_deployer.deploy_pytorch_model(
    clipper_conn,
    name="pytorch-model",
    version=1, 
    input_type="bytes", 
    func=predict_torch_model,
    pytorch_model=model,
)

In [ ]:
clipper_conn.link_model_to_app(app_name="squeezenet-classsifier", model_name="pytorch-model")

*Keeping track of metrics is an important service that Clipper offers users. Clipper reports metrics to its own Prometheus server - which can then be connected to other services, such as [Grafana](http://grafana.com). To help you visualize Clipper's metrics, this tutorial will use Grafana's API to connect Clipper's Prometheus server to Grafana and display metrics in a dashboard.*

The following cells will start up Grafana.

In [ ]:
this_ip = requests.get('http://ip.42.pl/raw').text
grafana_url = f"{this_ip}/camp/clipper/grafana/"

The following cell will register Clipper's Prometheus Server as a Data Source, and then import a custom dashboard for you to view your metrics with.

In [ ]:
from tutorial_utils import setup_grafana
dashboard_path = setup_grafana(grafana_url, metric_addr=clipper_conn.cm.get_metric_addr())

In [ ]:
dashboard_path

In [ ]:
from IPython.display import display, Image, Markdown
display(Markdown(f"""
This is the location of your Grafana instance:

<a href="http://{this_ip}{dashboard_path}">http://{this_ip}{dashboard_path}</a>

Please Login with username *admin* and password *admin*
"""))

You should see a dashboard titled `Clipper RISE Camp Tutorial Dashboard`. If not, Click `Clipper` under the Recently Viewed Dashboards tab to see the dashboard.

Now let send some predictions!

In [ ]:
import base64
import json
import requests

req_json = json.dumps({
        "input":
        base64.b64encode(open('images/cat.jpg', "rb").read()).decode() # bytes to unicode
    })

response = requests.post(
     "http://%s/%s/predict" % (clipper_addr, 'squeezenet-classsifier'),
     headers={"Content-type": "application/json"},
     data=req_json)
response.json()

> *Note that squeezenet is a deep-learning model (specifically, a convolutional neural network). Even though it was developed to be relatively fast, it may take a few seconds to return a prediction since it is running on a CPU. Deploying the model to a GPU will accelerate it substantially.*

<details>
<summary>*A Note on Batching*</summary>
<p>It's important to note that since we are sending requests in series, there aren't multiple requests in the system at any one time, and therefore the system will not have multiple queries to batch together. You will still be able to observe some batching, however, thanks to Clipper's batching exploration algorithm. Clipper will occasionally inject additional queries into the system (when it is underloaded) to explore the throughput-latency tradeoff for each model. This exploration algorithm estimates a function that predicts query latency based on the batch size, thereby enabling Clipper to choose the largest batch size (and therefore the highest throughput) that is still capable of returning within the deadline imposed by the latency SLO. Clipper estimates a different function for each model container, allowing the system to adapt to different model characteristics and varying resource loads.</p>
</details>

The following cells will download display images from the Internet, send them to the model, and then display the image and the result from the model. 

Here we show it in two flavors:
1. Send them one-by-one. 
2. Send them in a batch. This is expected to be faster because the model utilizes batch parallelism. 

Note that we add noise to image so prediction will not be cached.

Feel free to add the URLs of images to the `images` list to have those images be classified as well.

**TODO**: Please replace the ellipsis with your own URL's to run the following cell.

In [ ]:
from IPython.display import display, Image, Markdown
import time
from pprint import pprint
from tutorial_utils import _add_noise_to_img
import sys

# Blue is Rehan's cat's name. The last picture is an image of his cat.
images = ['https://raw.githubusercontent.com/ucbrise/clipper-tutorials/master/images/duck.jpg',
          'https://github.com/ucbrise/clipper-tutorials/raw/master/images/dog.jpg',
          'https://github.com/ucbrise/clipper-tutorials/raw/master/images/cat.jpg',
          'https://github.com/ucbrise/clipper-tutorials/raw/master/images/blue.jpg',
          'https://media.mnn.com/assets/images/2014/12/gray-squirrel-uc-berkeley.jpg.653x0_q80_crop-smart.jpg',
          'http://www3.pictures.zimbio.com/gi/Oski+Grambling+v+California+W8l7cBW82vyl.jpg',
          'https://people.eecs.berkeley.edu/~jegonzal/assets/jegonzal.jpg',
          'https://databricks.com/wp-content/uploads/2017/12/Ion-Stoica.jpg',
         ]
#           ...] # TODO: Add your own URL's.
    
total_time_ms = 0
for i, img in enumerate(images):
    img_bytes = _add_noise_to_img(requests.get(url=img).content)
    req_json = json.dumps({
            "input":
            base64.b64encode(img_bytes).decode() # bytes to unicode
        })

    begin_time = time.time()
    response = requests.post(
         "http://%s/%s/predict" % (clipper_addr, 'squeezenet-classsifier'),
         headers={"Content-type": "application/json"},
         data=req_json)
    end_time = time.time()

    duration_ms = (end_time-begin_time)*1000
    total_time_ms += duration_ms
    img_bytes = int(sys.getsizeof(img_bytes)/1024)
    display(Markdown(f"## > Image {i+1} ({img_bytes} kb) took {'{0:.3f}'.format(duration_ms)} miliseconds"))
    display(Image(url=img, width=200, height=200))
    pprint(response.json())

display(Markdown(f"## > Predicting {len(images)} images serially took {'{0:.3f}'.format(total_time_ms/1000)} seconds"))

In [ ]:
from IPython.display import display, Image, Markdown
import time
from pprint import pprint
import sys

display(Markdown(f"## > Now we will send these {len(images)} as a batch"))

start_time = time.time()
input_batch = [base64.b64encode(_add_noise_to_img(requests.get(url).content)).decode() for url in images]
end_time = time.time()
total_size = int(sum(map(sys.getsizeof, input_batch))/1024)
duration = end_time - start_time
display(Markdown(f"## > {total_size}kb downloaded in {'{0:.3f}'.format(duration)} s"))

req_json = json.dumps({
        "input_batch":
             input_batch
    })

start_time = time.time()
response = requests.post(
     "http://%s/%s/predict" % (clipper_addr, 'squeezenet-classsifier'),
     headers={"Content-type": "application/json"},
     data=req_json)
end_time = time.time()

duration_ms = end_time-start_time
display(Markdown(f"## > It takes {'{0:.3f}'.format(duration_ms)} seconds to predict this batch!"))
pprint(response.json())

### Example Application - Custom Docker Containers

In this final example, you will learn how to create a custom Docker model container. Custom Docker containers can be utilized when model containers rely on dependencies that cannot be installed with Pip. In this example, you will deploying a model developed in a pure C framework that must be compiled from source within the Docker container.

<details>
<summary>**Click here for more details on the Dockerfile and model you'll be using!**</summary>
<div class="text_cell_render rendered_html" tabindex="-1"><p>In this final example, you will learn how to create a custom Docker model container. Custom Docker containers can be utilized when model containers rely on dependencies that cannot be installed with Pip. In this example, you will deploying a model developed in a pure C framework that must be compiled from source within the Docker container.</p>
<p>You will deploy the <a href="https://pjreddie.com/darknet/yolo/" target="_blank">YoloV3 Real Time Object Detection System</a> to predict both <em>labels and bounding boxes</em> for objects within an image. This is a harder machine learning task than the image classification task that you performed with SqueezeNet.</p>
<p>YOLO (You Only Look Once) is a state-of-the-art object detection model, but unfortunately it does not expose a native Python API. Instead, it must be compiled from source and executed as a binary executable, not as a library. You will clone the repo (in this case, a fork we created with some minor modifications to extract the bounding box coordinates instead of directly drawing the on the image) and compile it, all within the Docker image.</p>
<p>The first step in creating a Docker image is to write a <a href="https://docs.docker.com/engine/reference/builder/" target="_blank">Dockerfile</a>. This is like the source code for the image. After you have written the Dockerfile, you will build it. Unlike when you compile source code, building a Docker image actually runs the program specified in the Dockerfile. The end result is a Docker image -- a binary object that contains the results of executing the instructions in the Dockerfile sequentially from top to bottom. </p>
<p>For your convenience, we have already written a Dockerfile for you called <code>PyDarknetDockerfile</code> and included it in the tutorial. However, before you build it you will read about each step in the Dockerfile, providing a blueprint for you to define your own custom Clipper model containers in the future.</p>
<p>Here are the full contents of the Dockerfile:</p>
<pre><code class="cm-s-ipython language-dockerfile"><span class="cm-variable-2" style="color:green"><strong>FROM</strong></span> clipper/python36-closure-container:0.3

<span class="cm-comment"># Install Git</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> apt-get update \
    &amp;&amp; apt-get install -y git

<span class="cm-comment"># Install cURL</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> apt-get install -y curl

<span class="cm-comment"># Clone Darknet Repo</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> git clone https://github.com/RehanSD/darknet.git /tmp/darknet
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> mv /tmp/darknet/* .

<span class="cm-comment"># Make Darknet Project</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> make -j16

<span class="cm-comment">#Download Weights</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> curl -o yolov3-tiny.weights https://pjreddie.com/media/files/yolov3-tiny.weights
</code></pre>
<p>In the following sections, we will break down each part of the file.</p>
</div>
<div class="text_cell_render rendered_html" tabindex="-1"><p>The first line of the file ensures that you are starting off with a Docker image with all of the necessary Clipper dependencies installed. In the rest of the Dockerfile, you are extending the Clipper-provided container to simply install some additional dependencies.</p>
<pre><code class="cm-s-ipython language-dockerfile"><span class="cm-variable-2" style="color:green"><strong>FROM</strong></span> clipper/python36-closure-container:0.3
</code></pre>
<p>The next few lines install the basic dependencies we need in the container - git to clone the repo, annd curl to download the pretrained yolo model weights.</p>
<pre><code class="cm-s-ipython language-dockerfile"><span class="cm-comment"># Install Git</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> apt-get update \
    &amp;&amp; apt-get install -y git

<span class="cm-comment"># Install cURL</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> apt-get install -y curl
</code></pre>
<p>The last few lines clone the darknet repo, compile it, and then download the pretrained model weights. We are using <code>yolov3-tiny</code> because it offers a good trade off between accuracy and inference speed on a CPU. (For a latency-sensitive production use case, you might run it on GPU.)</p>
<pre><code class="cm-s-ipython language-dockerfile"><span class="cm-comment"># Clone Darknet Repo</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> git clone https://github.com/RehanSD/darknet.git /tmp/darknet
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> mv /tmp/darknet/* .

<span class="cm-comment"># Make Darknet Project</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> make -j16

<span class="cm-comment">#Download Weights</span>
<span class="cm-variable-2" style="color:green"><strong>RUN</strong></span> curl -o yolov3-tiny.weights https://pjreddie.com/media/files/yolov3-tiny.weights
</code></pre>
<p>Now that you have defined the image, you can build it with the following shell command.</p>
</div>
</details>

In [ ]:
!docker build -t clipper/darknet-yolov3-container -f PyDarknetDockerfile .

#### Predict Function
Now we build the predict function. We must first deserialize the image that we serialized in our request - similar to how we did in the PyTorch example, and then write it to a file to run darknet on it. Since darknet is a C executable, we must call it using the subprocess API. It is not strictly neccessary to print out the output of calling darknet, but it is useful to have the output for the sake of debugging.

In [ ]:
def yolo_pred(imgs):
    import base64
    import io
    import os
    import tempfile
    import subprocess

    num_imgs = len(imgs)
    ret_coords = []
    predict_procs = []
    file_names = []
    
    # First, we save the images to file
    for i in range(num_imgs):
        # Create a temp file to write to
        tmp = tempfile.NamedTemporaryFile('wb', delete=False, suffix='.jpg')
        tmp.write(io.BytesIO(imgs[i]).getvalue())
        tmp.close()
        file_names.append(tmp.name)
    
    # Second, we call ./darknet executable to detect objects in images.
    #   This is done in parallel.
    for file_name in file_names:
        process = subprocess.Popen(
            ['./darknet',
             'detector',
             'test',
             './cfg/coco.data',
             './cfg/yolov3-tiny.cfg',
             './yolov3-tiny.weights',
             file_name,
             '-json',
             '-dont_show',
             '-ext_output', '>',
             '{}.txt'.format(file_name+'_result')], stdout=subprocess.PIPE)
        predict_procs.append(process)
        
    # Lastly, we wait for all process to finished and return stdout of each process
    for process in predict_procs:
        process.wait()
        ret_coords += [' '.join(map(lambda byte_str: byte_str.decode(), process.stdout))]

    return ret_coords

In [ ]:
# Do not be concerned if this cell takes a couple of seconds to run.
from clipper_admin.deployers import python as python_deployer
python_deployer.deploy_python_closure(
    clipper_conn,
    name="yolov3",  # The name of the model in Clipper
    version=1,  # A unique identifier to assign to this model.
    input_type="bytes",  # The type of data the model function expects as input
    func=yolo_pred, # The model function to deploy
    base_image='clipper/darknet-yolov3-container'
)

In [ ]:
clipper_conn.register_application(
    name="darknet-app",
    input_type="bytes",
    default_output="Default",
    slo_micros=10000000 # 10 seconds
)

In [ ]:
clipper_conn.link_model_to_app(app_name="darknet-app", model_name="yolov3")

In [ ]:
# Please note the request may take a couple of seconds to return, as we are running a largeish model
# on a CPU, which is slow.
import requests, json
from pprint import pprint

url = "http://%s/darknet-app/predict" % clipper_addr
req_json = json.dumps({
    "input":
    base64.b64encode(open('images/dog.jpg', "rb").read()).decode() # bytes to unicode
})
headers = {'Content-type': 'application/json'}
r = requests.post(url, headers=headers, data=req_json)

# Let's see what does YoloV3-tiny return
pprint(r.json())

In [ ]:
# Let's plot the result
from tutorial_utils import plot_bbox, predict_and_plot_url, predict_and_plot
%matplotlib inline
plot_bbox('images/dog.jpg', r.json()['output'])

Feel free to experiment with other images under `images/*.jpg`, or to use the `predict_and_plot_url` function with URLs of different images!

In [ ]:
!ls images/

In [ ]:
# Please note the request may take a couple of seconds to return, as we are running a largeish model
# on a CPU, which is slow.
# im_name = "images/detection/*.jpg"
im_name = "images/cityscapes-3.jpg"
predict_and_plot(im_name, clipper_addr)
predict_and_plot_url('https://github.com/ucbrise/clipper-tutorials/raw/master/images/detection/baseball-boy.jpg', clipper_addr)

Now that we've sent some images, you should check out Grafana again to see some interesting metrics!

In [ ]:
from IPython.display import display, Image, Markdown
display(Markdown(f"""
This is the location of your Grafana instance:

<a href="http://{this_ip}{dashboard_path}">http://{this_ip}{dashboard_path}</a>

Please Login with username *admin* and password *admin*
"""))

### Stopping Clipper
If you run into issues and want to completely stop Clipper, you can do this by calling [`ClipperConnection.stop_all()`](http://docs.clipper.ai/en/latest/#clipper_admin.ClipperConnection.stop_all).

Uncomment the following cell to complete stop Clipper.

In [ ]:
# clipper_conn.stop_all()

When you list all the Docker containers a final time, you should see that all of the Clipper containers have been stopped.

In [ ]:
!docker ps --filter label=ai.clipper.container.label

You can now call `clipper_conn.start_clipper()` again without running into errors.

Thanks for participating in this tutorial! Please fill the [feedback form](https://goo.gl/forms/gwPcOWRdnbIaWv1L2) to tell us your interests in Clipper. 